In [ ]:
library("Seurat")
library("tidyverse")
library('harmony')
library('presto')
library("scDblFinder")
library('miloR')
library("cowplot")
library("viridis")
library('cetcolor')

In [ ]:
library('msigdbr')
library('fgsea')

In [ ]:
sessionInfo()

In [ ]:
library('ggpubr')

In [ ]:
wd <- "/public/home/liwang/project/xu_lab/Qinyue_xuguotai_CD45_20241216"

In [ ]:

# ltc_palettes package
main_celltype_colors <- c('CD8T' = '#9b2226', 'CD4T' = '#D55D4C', 'gdT' = '#ca6702', 'NKT' = '#ee9b00', 'NK' = '#C2C1E0',
                          'Macrophage' = '#005F73', 'Monocyte' = '#0a9396', 'cDC1' = '#94d2bd', 'cDC2' = '#66679C', 'Neutrophil' = '#e9d8a6',
                          'mregDC' = '#B2CAEE', 'pDC' = '#D17C7D', 'Basophil' = '#F3BAA5', 'B_Cell' = '#67ADB7')

#'NK' = '#f897a1'

In [ ]:
imm_subcelltype_colors <- c(Memory_B = "#2d6037", Naive_B = "#64AE59", Plasma_B = "#839098", CD8T_Tn = "#ECA8A9",
                            GZMK_CD8_Tem = "#74AED4", CD8T_Tstem = "#67ADB7",CXCR6_CD4_Trm = "#E4A6BD", ISG15_Teffector = '#B3B2B3',
                            MAIT = "#F3D8E1", FOXP3_CD4_Treg = "#009170", CXCL13_CD4_Tex = "#78A040", IFITM3_CD8_Teffector = '#839098',
                            CD8T_Teffector = "#2E75AB", CD8T_Ttd = "#009393",FGFBP2_NK = "#B06E3C", XCL1_NK = "#5AA2DA",
                            NKT = "#FBD8A2", ILC = "#B84848", Mast = "#6567A0", Undetermined = "#87C3EC",
                            pDC = "#BDE6FA", cDC1 = "#D7EFFB", cDC2 = "#6EB1DE", LAMP3_DC = "#92C2DD", Neutrophil = "#4A94C6",
                            PPARG_Mono = "#FADED2", CD16_Mono = "#FAC7B3", PPARG_Macro = "#F0A29B", SPP1_Macro = "#B389B9",
                            LGMN_Macro = "#CC7892", FABP4_Macro = "#E2A2B3", CD14_Mono = "#F3C6C1", MIF_Macro = "#89558D")

#c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd')

In [ ]:
transparent_bg <- theme(panel.background = element_rect(fill = NA, colour = NA),
                        plot.background = element_rect(fill = NA, colour = NA),
                        legend.box.background = element_rect(fill = NA, colour = NA),
                        legend.background = element_rect(fill = NA, colour = NA),
                        strip.background = element_rect(fill = NA, colour = NA))

In [ ]:
sgRNA_color <- c('sgNT' = 'gray', 'sgC' = '#ee6378', 'sgS' = '#b63f53')

In [ ]:
sgRNA_color <- c('sgNT' = '#808080', 'sgC' = '#FF8080', 'sgS' = '#FF0000')

# scRNA Integrated analysis

## Data Load

In [ ]:
##load the B16F10 data from filtered_feature_bc_matrix (10x platform)
Cd45_samples <-c("NT", "C", "S")

#
for (sample in Cd45_samples) {
    print(paste0(wd, '/data/', sample,"/filtered_feature_bc_matrix"))
    
    assign(paste0("Cd45_", sample, "_matrix"), 
           Read10X(data.dir = paste0(wd, '/data/', sample,"/filtered_feature_bc_matrix")))
    
    assign(paste0("Cd45_", sample), 
           CreateSeuratObject(get(paste0("Cd45_", sample, "_matrix")), project = paste0("Cd45_", sample)))
}

In [ ]:
##load the B16F10 data  (BD platform, mm10)
Cd45_BD_SeuratObj <- readRDS(paste0(wd, '/data/rhapsodyPipeline_mm10/250115-B16-SMK_1_Seurat.rds'))

# subset the cells belong to SampleTag10_mm, SampleTag11_mm, SampleTag12_mm
Cd45_BD_SeuratObj <- subset(Cd45_BD_SeuratObj, subset = Sample_Tag %in% c('SampleTag10_mm', 'SampleTag11_mm', 'SampleTag12_mm'))
Cd45_BD_SeuratObj$`orig.ident` <- Cd45_BD_SeuratObj$`Sample_Tag`
Cd45_BD_SeuratObj$`Sample_Tag` <- NULL
Cd45_BD_SeuratObj$`Sample_Name` <- NULL

In [ ]:
Cd45_BD_SeuratObj

In [ ]:
##load the MC38 data from filtered_feature_bc_matrix (10x platform)
Cd45_samples <-c("HXF_CC", "HXF_NT", "JFF_NT", "JFF_CC")

#
for (sample in Cd45_samples) {
    print(paste0(wd, '/data/', sample,"/filtered_feature_bc_matrix"))
    
    assign(paste0("Cd45_", sample, "_matrix"), 
           Read10X(data.dir = paste0(wd, '/data/', sample,"/filtered_feature_bc_matrix")))
    
    assign(paste0("Cd45_", sample), 
           CreateSeuratObject(get(paste0("Cd45_", sample, "_matrix")), project = paste0("Cd45_", sample)))
}

In [ ]:
ls()

In [ ]:
Cd45_NT;
Cd45_C;
Cd45_S;
Cd45_BD_SeuratObj;
Cd45_HXF_NT;
Cd45_HXF_CC;
Cd45_JFF_NT;
Cd45_JFF_CC;

In [ ]:
# merge all into one object 
SAGA_TIL_SeuratObj_List <- list(Cd45_NT, Cd45_C, Cd45_S, Cd45_BD_SeuratObj, Cd45_HXF_NT, Cd45_HXF_CC, Cd45_JFF_NT, Cd45_JFF_CC)
common_genes <- Reduce(intersect, lapply(SAGA_TIL_SeuratObj_List, rownames))
SAGA_TIL_SeuratObj_List <- lapply(SAGA_TIL_SeuratObj_List, function(x) x[common_genes, ])

SAGA_TIL_SeuratObj <- merge(SAGA_TIL_SeuratObj_List[[1]], y = SAGA_TIL_SeuratObj_List[-1])

In [ ]:
SAGA_TIL_SeuratObj$orig.ident %>% unique()

In [ ]:
head(SAGA_TIL_SeuratObj[[]])

In [ ]:
# harmonize the metadata

SAGA_TIL_Metadata <- SAGA_TIL_SeuratObj[[]] %>% 
    mutate(sgRNA = case_match(orig.ident,
                              c("Cd45_NT", "SampleTag10_mm", "Cd45_HXF_NT", "Cd45_JFF_NT") ~ "sgNT",
                              c("Cd45_C", "SampleTag11_mm", "Cd45_HXF_CC", "Cd45_JFF_CC") ~ "sgC",
                              c("Cd45_S", "SampleTag12_mm") ~ "sgS")) %>%
    mutate(CellLine = case_match(orig.ident,
                                 c("Cd45_NT", "Cd45_C", "Cd45_S", "SampleTag10_mm", "SampleTag11_mm", "SampleTag12_mm") ~ "B16F10",
                                 c("Cd45_HXF_NT", "Cd45_JFF_NT", "Cd45_HXF_CC", "Cd45_JFF_CC") ~ "MC38")) %>%
    mutate(ExperimentID = case_match(orig.ident,
                                     c("Cd45_NT", "Cd45_C", "Cd45_S") ~ "Experiment1",
                                     c("SampleTag10_mm", "SampleTag11_mm", "SampleTag12_mm") ~ "Experiment2",
                                     c("Cd45_HXF_NT", "Cd45_HXF_CC") ~ "Experiment3",
                                     c("Cd45_JFF_NT", "Cd45_JFF_CC") ~ "Experiment4")) %>%
    mutate(Platform = case_match(orig.ident,
                                 c("Cd45_NT", "Cd45_C", "Cd45_S", "Cd45_HXF_NT", "Cd45_JFF_NT", "Cd45_HXF_CC", "Cd45_JFF_CC") ~ "10x",
                                 c("SampleTag10_mm", "SampleTag11_mm", "SampleTag12_mm") ~ "BD")) %>%
    mutate(Operator = case_match(orig.ident,
                                 c("Cd45_NT", "Cd45_C", "Cd45_S", "SampleTag10_mm", "SampleTag11_mm", "SampleTag12_mm") ~ "QY",
                                 c("Cd45_HXF_NT", "Cd45_HXF_CC") ~ "HXF",
                                 c("Cd45_JFF_NT", "Cd45_JFF_CC") ~ "JFF")) %>%
    unite(Sample, c('CellLine', 'sgRNA', 'ExperimentID'), remove = F) %>%
    mutate(sgRNA = factor(sgRNA, levels = c('sgNT', 'sgC', 'sgS')))
    

SAGA_TIL_SeuratObj <- AddMetaData(object = SAGA_TIL_SeuratObj, metadata = SAGA_TIL_Metadata)

In [ ]:
SAGA_TIL_SeuratObj$Sample %>% unique()

In [ ]:
dim(SAGA_TIL_SeuratObj)

## Delete doublet cell

In [ ]:
SAGA_TIL_SeuratObj$nCount_RNA %>% summary()

In [ ]:
#remove cells with a very low coverage (e.g. <200 reads)
SAGA_TIL_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = nCount_RNA > 200)

In [ ]:
SAGA_TIL_SeuratObj_List <- SplitObject(object = SAGA_TIL_SeuratObj, split.by = 'Sample')

In [ ]:
### determine the multiplets using scDblFinder (the dataset does not contain any empty drops, but hasn't been further filtered; be necessary to remove cells with a very low coverage (e.g. <200 reads) to avoid errors)


SAGA_TIL_sce_List <- lapply(X = SAGA_TIL_SeuratObj_List, FUN = function (x) {
    
    x <- as.SingleCellExperiment( x, assay = "RNA")
    
    set.seed(1234)
    x <- scDblFinder(x)

})

In [ ]:
saveRDS(SAGA_TIL_sce_List, paste0(wd, '/objects_v2/SAGA_TIL_sce_List.rds'))

In [ ]:
### subset the singlet
SAGA_TIL_singlet_list <- lapply(X = SAGA_TIL_sce_List, FUN = function (x) {
    
    print(table(x$scDblFinder.class))
    x <- rownames(subset(colData(x), scDblFinder.class == "singlet"))
    
})

### subset the singlet
for ( i in seq_along(SAGA_TIL_SeuratObj_List)) {
    SAGA_TIL_SeuratObj_List[[i]] <- subset(SAGA_TIL_SeuratObj_List[[i]], cells = SAGA_TIL_singlet_list[[names(SAGA_TIL_SeuratObj_List[i])]])
}

In [ ]:
n_cells <- 0
for ( i in SAGA_TIL_SeuratObj_List) {
    n_cells <- nrow(i[[]]) + n_cells
}

n_cells

In [ ]:
# merge all into one object
SAGA_TIL_SeuratObj <- merge(x = SAGA_TIL_SeuratObj_List[[1]], y = SAGA_TIL_SeuratObj_List[-1])

In [ ]:
SAGA_TIL_SeuratObj %>% dim()

## Quality control

In [ ]:
# data qc: calculate pMT, pHB

SAGA_TIL_SeuratObj <- PercentageFeatureSet(SAGA_TIL_SeuratObj, pattern = "^mt-", col.name = "pMT")
SAGA_TIL_SeuratObj <- PercentageFeatureSet(SAGA_TIL_SeuratObj, pattern = "^Hba|^Hbb", col.name = "pHB")

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 10)
VlnPlot(object = SAGA_TIL_SeuratObj, features = c('pMT', 'pHB'), pt.size = 0, ncol = 1, group.by = 'Sample')

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 10)
VlnPlot(object = SAGA_TIL_SeuratObj, features = c('nCount_RNA', 'nFeature_RNA'), pt.size = 0, ncol = 1, group.by = 'Sample') +
    geom_hline(yintercept = 1000, linetype = "dashed", color = "red")

In [ ]:
#qc parameters
nFeature_lower <- 200        # the lower bound is set as 200
nFeature_upper <- 6500
nCount_lower <- 1000
nCount_upper <- Inf
pMT_lower <- 0
pMT_upper <- 10
pHB_lower <- 0
pHB_upper <- 5

In [ ]:
# filter based on n_feature, n_count, pMT, pHB

SAGA_TIL_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = nFeature_RNA > nFeature_lower & nFeature_RNA < nFeature_upper & nCount_RNA > nCount_lower & nCount_RNA < nCount_upper & pMT < pMT_upper & pHB < pHB_upper)

In [ ]:
dim(SAGA_TIL_SeuratObj)

In [ ]:
SAGA_TIL_SeuratObj

In [ ]:
saveRDS(SAGA_TIL_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_SeuratObj.rds'))

## Cell line subset and integration

In [ ]:
SAGA_TIL_SeuratObj <- readRDS(paste0(wd, '/objects_v2/SAGA_TIL_SeuratObj.rds'))

In [ ]:
#SAGA_TIL_B16_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = CellLine == 'B16F10' & Maincluster_res0.3 != 'Unknown' & Maincluster_res0.3 != 'Stromal' & Maincluster_res0.3 != 'Low_Quality')
#SAGA_TIL_MC38_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = CellLine == 'MC38' & Maincluster_res0.3 != 'Unknown' & Maincluster_res0.3 != 'Stromal' & Maincluster_res0.3 != 'Low_Quality')

In [ ]:
SAGA_TIL_B16_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = CellLine == 'B16F10')
SAGA_TIL_MC38_SeuratObj <- subset(SAGA_TIL_SeuratObj, subset = CellLine == 'MC38')

### B16F10

In [ ]:
#normalization
SAGA_TIL_B16_SeuratObj <- SAGA_TIL_B16_SeuratObj %>% NormalizeData(verbose = FALSE)

In [ ]:
# find variable genes in each sample
seurat_list <- SplitObject(SAGA_TIL_B16_SeuratObj, split.by = "Sample")


seurat_list <- lapply(seurat_list, function(obj) {
  obj <- NormalizeData(obj)
  obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000)
  return(obj)
})

# 
fvf_features <- SelectIntegrationFeatures(object.list = seurat_list, nfeatures = 2000)

# 
VariableFeatures(SAGA_TIL_B16_SeuratObj) <- fvf_features

In [ ]:
#run scale_data
SAGA_TIL_B16_SeuratObj <- ScaleData(SAGA_TIL_B16_SeuratObj, verbose = T)

In [ ]:
SAGA_TIL_B16_SeuratObj

In [ ]:
SAGA_TIL_B16_SeuratObj <- RunPCA(SAGA_TIL_B16_SeuratObj, features = as.character(VariableFeatures(SAGA_TIL_B16_SeuratObj)), npcs = 50, verbose = FALSE)

In [ ]:
# run harmony
options(repr.plot.height = 8, repr.plot.width = 8)
SAGA_TIL_B16_SeuratObj <- RunHarmony(object = SAGA_TIL_B16_SeuratObj, group.by.vars = 'Sample', max.iter.harmony = 20, plot_convergence = T)

In [ ]:
# UMAP
SAGA_TIL_B16_SeuratObj <- RunUMAP(SAGA_TIL_B16_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F )

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 10)
DimPlot(object = SAGA_TIL_B16_SeuratObj, reduction = 'umap', group.by = 'Sample')

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 10)
DimPlot(object = SAGA_TIL_B16_SeuratObj, reduction = 'umap', group.by = 'CellLine')

### MC38

In [ ]:
#normalization
SAGA_TIL_MC38_SeuratObj <- SAGA_TIL_MC38_SeuratObj %>% NormalizeData(verbose = FALSE)

In [ ]:
# find variable genes in each sample
seurat_list <- SplitObject(SAGA_TIL_MC38_SeuratObj, split.by = "Sample")


seurat_list <- lapply(seurat_list, function(obj) {
  obj <- NormalizeData(obj)
  obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000)
  return(obj)
})

# 
fvf_features <- SelectIntegrationFeatures(object.list = seurat_list, nfeatures = 2000)

# 
VariableFeatures(SAGA_TIL_MC38_SeuratObj) <- fvf_features

In [ ]:
#run scale_data
SAGA_TIL_MC38_SeuratObj <- ScaleData(SAGA_TIL_MC38_SeuratObj, verbose = T)

In [ ]:
SAGA_TIL_MC38_SeuratObj

In [ ]:
SAGA_TIL_MC38_SeuratObj <- RunPCA(SAGA_TIL_MC38_SeuratObj, features = as.character(VariableFeatures(SAGA_TIL_MC38_SeuratObj)), npcs = 50, verbose = FALSE)

In [ ]:
# run harmony
options(repr.plot.height = 8, repr.plot.width = 8)
SAGA_TIL_MC38_SeuratObj <- RunHarmony(object = SAGA_TIL_MC38_SeuratObj, group.by.vars = 'Sample', max.iter.harmony = 20, plot_convergence = T)

In [ ]:
# UMAP
SAGA_TIL_MC38_SeuratObj <- RunUMAP(SAGA_TIL_MC38_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F )

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 10)
DimPlot(object = SAGA_TIL_MC38_SeuratObj, reduction = 'umap', group.by = 'Sample')

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 10)
DimPlot(object = SAGA_TIL_MC38_SeuratObj, reduction = 'umap', group.by = 'CellLine')

In [ ]:
saveRDS(SAGA_TIL_B16_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_B16_SeuratObj.rds'))
saveRDS(SAGA_TIL_MC38_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_MC38_SeuratObj.rds'))

## Clustering and annotation

### B16F10

In [ ]:
SAGA_TIL_B16_SeuratObj

In [ ]:
# FindNeighbors
SAGA_TIL_B16_SeuratObj <- FindNeighbors(SAGA_TIL_B16_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindClusters
options(repr.plot.height = 8, repr.plot.width = 8)
for (i in c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1)) {
  SAGA_TIL_B16_SeuratObj <- FindClusters(SAGA_TIL_B16_SeuratObj, resolution = i, verbose = F)
  print(DimPlot(SAGA_TIL_B16_SeuratObj, reduction = "umap", label = T) + labs(title = paste0("resolution: ", i)))
}

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 42)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67') #Prolif.

FeaturePlot(object = SAGA_TIL_B16_SeuratObj, features = mainmarkers[1:25], order = F) *
    theme(plot.title = element_text(size = 25))

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 42)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67') #Prolif.

FeaturePlot(object = SAGA_TIL_B16_SeuratObj, features = mainmarkers[26:length(mainmarkers)], order = F) *
    theme(plot.title = element_text(size = 25))

In [ ]:
# fine-grain subclustering 

In [ ]:
# get the multiple_subcluster for c0_res0.3
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$`RNA_snn_res.0.3`
SAGA_TIL_B16_SeuratObj <- FindSubCluster(object = SAGA_TIL_B16_SeuratObj, cluster = '0', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
# get the multiple_subcluster for c0_1_res0.3
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$'RNA_snn_res.0.3_subclus'
SAGA_TIL_B16_SeuratObj <- FindSubCluster(object = SAGA_TIL_B16_SeuratObj, cluster = '0_1', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.2)


In [ ]:
# get the multiple_subcluster for c4_res0.3
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$'RNA_snn_res.0.3_subclus'
SAGA_TIL_B16_SeuratObj <- FindSubCluster(object = SAGA_TIL_B16_SeuratObj, cluster = '4', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
# get the multiple_subcluster for c5_res0.3
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$'RNA_snn_res.0.3_subclus'
SAGA_TIL_B16_SeuratObj <- FindSubCluster(object = SAGA_TIL_B16_SeuratObj, cluster = '5', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
DimPlot(SAGA_TIL_B16_SeuratObj, reduction = "umap", label = T, label.size = 8, repel = T,group.by = 'RNA_snn_res.0.3_subclus')

In [ ]:
#find Mainmarkers_B16_res0.3_presto with wilcox test in presto
Mainmarkers_B16_res0.3_presto <- wilcoxauc(X = SAGA_TIL_B16_SeuratObj, group_by = 'RNA_snn_res.0.3_subclus')

In [ ]:
Mainmarkers_B16_res0.3_presto %>% filter(group=='0_1_2', pct_in > 10) %>% arrange(desc(logFC)) %>% slice_max(logFC, n = 60)

In [ ]:
# Mannual annotation
options(repr.plot.height = 10, repr.plot.width = 20)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67', #Prolif.
                 'Col3a1', 'Dcn')

DotPlot(object = SAGA_TIL_B16_SeuratObj, features = mainmarkers, group.by = 'RNA_snn_res.0.3_subclus') +
    coord_flip()

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
DimPlot(SAGA_TIL_B16_SeuratObj, reduction = "umap", label = T, label.size = 8, repel = T,group.by = 'RNA_snn_res.0.3_subclus')

In [ ]:
as.character(SAGA_TIL_B16_SeuratObj$`RNA_snn_res.0.3_subclus`) %>% unique() %>% length()

In [ ]:
##maincluster RNA_snn_res.0.3
options(repr.plot.height = 10, repr.plot.width = 12)


maincluster_anno <- c('0_0' = 'CD8T', '0_1_0' = 'CD4T', '0_1_1' = 'CD8T', '0_1_2' = 'CD4T', '0_2' = 'CD8T',
                      '1' = 'Monocyte', '2' = 'NK', '3' = 'Monocyte', '4_0' = 'NKT', '4_1' = 'CD8T', '4_2' = 'NKT', '4_3' = 'NKT',
                      '5_0' = 'CD8T', '5_1' = 'CD8T', '5_2' = 'NK', '5_3' = 'CD8T',
                      '6' = 'cDC2', '7' = 'CD4T', '8' = 'Macrophage',
                      '9' = 'mregDC', '10' = 'cDC1', '11' = 'pDC', '12' = 'Neutrophil', '13' = 'B_Cell', '14' = 'gdT', '15' = 'Basophil')




SAGA_TIL_B16_SeuratObj$Maincluster_res0.3 <- maincluster_anno[ as.character(SAGA_TIL_B16_SeuratObj$`RNA_snn_res.0.3_subclus`) ]


DimPlot(SAGA_TIL_B16_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Maincluster_res0.3') +
    theme(plot.title = element_text(size = 30),
          legend.text = element_text(size = 20),
          legend.key.size = unit(0.5, "inches")) +
    guides(colour = guide_legend(override.aes = list(size = 5)))

In [ ]:
SAGA_TIL_B16_SeuratObj <- subset(SAGA_TIL_B16_SeuratObj, subset = CellLine == 'B16F10' & Maincluster_res0.3 != 'Unknown' & Maincluster_res0.3 != 'Stromal' & Maincluster_res0.3 != 'Low_Quality')

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 12)
DimPlot(SAGA_TIL_B16_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Maincluster_res0.3') +
    scale_color_manual(values = main_celltype_colors, limits = c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')) +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 12)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Mki67' #Prolif.
                )

DotPlot(SAGA_TIL_B16_SeuratObj, features = mainmarkers, group.by = 'Maincluster_res0.3', scale = T) + 
    scale_color_gradient(low = '#fde9eb', high = '#db2531') + #FF0000 #db2531
#    scale_size_area(max_size = 1) +
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 90),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_MarkerGene_DotPlot.pdf'), device = 'pdf', width = 15, height = 6, bg = 'transparent')

In [ ]:
SAGA_TIL_B16_SeuratObj

In [ ]:
saveRDS(SAGA_TIL_B16_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_B16_SeuratObj.rds'))

### MC38

In [ ]:
SAGA_TIL_MC38_SeuratObj

In [ ]:
# FindNeighbors
SAGA_TIL_MC38_SeuratObj <- FindNeighbors(SAGA_TIL_MC38_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindClusters
options(repr.plot.height = 8, repr.plot.width = 8)
for (i in c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1)) {
  SAGA_TIL_MC38_SeuratObj <- FindClusters(SAGA_TIL_MC38_SeuratObj, resolution = i, verbose = F)
  print(DimPlot(SAGA_TIL_MC38_SeuratObj, reduction = "umap", label = T) + labs(title = paste0("resolution: ", i)))
}

In [ ]:
saveRDS(SAGA_TIL_MC38_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_MC38_SeuratObj.rds'))

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 42)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67') #Prolif.

FeaturePlot(object = SAGA_TIL_MC38_SeuratObj, features = mainmarkers[1:25], order = F) *
    theme(plot.title = element_text(size = 25))

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 42)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67') #Prolif.

FeaturePlot(object = SAGA_TIL_MC38_SeuratObj, features = mainmarkers[26:length(mainmarkers)], order = F) *
    theme(plot.title = element_text(size = 25))

In [ ]:
# fine-grain subclustering 

In [ ]:
# get the multiple_subcluster for c6_res0.3
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$`RNA_snn_res.0.3`
SAGA_TIL_MC38_SeuratObj <- FindSubCluster(object = SAGA_TIL_MC38_SeuratObj, cluster = '6', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
# get the multiple_subcluster for c7_res0.3
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$'RNA_snn_res.0.3_subclus'
SAGA_TIL_MC38_SeuratObj <- FindSubCluster(object = SAGA_TIL_MC38_SeuratObj, cluster = '7', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
# get the multiple_subcluster for c8_res0.3
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$'RNA_snn_res.0.3_subclus'
SAGA_TIL_MC38_SeuratObj <- FindSubCluster(object = SAGA_TIL_MC38_SeuratObj, cluster = '8', subcluster.name = 'RNA_snn_res.0.3_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
DimPlot(SAGA_TIL_MC38_SeuratObj, reduction = "umap", label = T, label.size = 8, repel = T,group.by = 'RNA_snn_res.0.3_subclus')

In [ ]:
#find Mainmarkers_MC38_res0.3_presto with wilcox test in presto
Mainmarkers_MC38_res0.3_presto <- wilcoxauc(X = SAGA_TIL_MC38_SeuratObj, group_by = 'RNA_snn_res.0.3_subclus')

In [ ]:
Mainmarkers_MC38_res0.3_presto %>% filter(group=='6_2', pct_in > 10) %>% arrange(desc(logFC)) %>% slice_max(logFC, n = 60)

In [ ]:
# Loss of ADAR1 in tumours overcomes resistance to immune checkpoint blockade
options(repr.plot.height = 8, repr.plot.width = 20)

mainmarkers <- c('Itgam', 'Adgre1', 'C1qa', 'Apoe', #universe
                 'Cxcl9', 'H2-Aa', 'Cd74', 'Cd86', #M1
                 'Mrc1', 'Folr2', 'Sepp1', 'Maf', #M2
                 'Arg1', 'Nos2', 'Pf4', 'Fabp5', 'Il1rn') #MDSC


DotPlot(object = SAGA_TIL_MC38_SeuratObj, features = mainmarkers, group.by = 'RNA_snn_res.0.3_subclus') +
#    scale_y_discrete(limits = factor(seq(0,21))) +
    coord_flip()

In [ ]:
# Mannual annotation
options(repr.plot.height = 10, repr.plot.width = 20)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Sell", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", "Nr4a1", "F13a1", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 'Mki67', #Prolif.
                 'Col3a1', 'Dcn')

DotPlot(object = SAGA_TIL_MC38_SeuratObj, features = mainmarkers, group.by = 'RNA_snn_res.0.3_subclus') +
    coord_flip()

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 15)
DimPlot(SAGA_TIL_MC38_SeuratObj, reduction = "umap", label = T, label.size = 8, repel = T,group.by = 'RNA_snn_res.0.3_subclus')

In [ ]:
as.character(SAGA_TIL_MC38_SeuratObj$`RNA_snn_res.0.3_subclus`) %>% unique() %>% length()

In [ ]:
##maincluster RNA_snn_res.0.3
options(repr.plot.height = 10, repr.plot.width = 12)


maincluster_anno <- c('0' = 'Monocyte', '1' = 'Macrophage', '2' = 'Macrophage', '3' = 'Macrophage', '4' = 'Macrophage', '5' = 'Macrophage',
                      '6_0' = 'CD8T', '6_1' = 'CD4T', '6_2' = 'CD8T', '6_3' = 'CD8T', '6_4' = 'gdT',
                      '7_0' = 'Monocyte', '7_1' = 'Monocyte', '7_2' = 'Neutrophil', '8_0' = 'cDC2', '8_1' = 'cDC1', '8_2' = 'pDC',
                      '9' = 'NK', '10' = 'mregDC', '11' = 'Monocyte', '12' = 'Stromal', '13' = 'Basophil')




SAGA_TIL_MC38_SeuratObj$Maincluster_res0.3 <- maincluster_anno[ as.character(SAGA_TIL_MC38_SeuratObj$`RNA_snn_res.0.3_subclus`) ]


DimPlot(SAGA_TIL_MC38_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Maincluster_res0.3') +
    theme(plot.title = element_text(size = 30),
          legend.text = element_text(size = 20),
          legend.key.size = unit(0.5, "inches")) +
    guides(colour = guide_legend(override.aes = list(size = 5)))

In [ ]:
SAGA_TIL_MC38_SeuratObj <- subset(SAGA_TIL_MC38_SeuratObj, subset = CellLine == 'MC38' & Maincluster_res0.3 != 'Unknown' & Maincluster_res0.3 != 'Stromal' & Maincluster_res0.3 != 'Low_Quality')

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 12)
DimPlot(SAGA_TIL_MC38_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Maincluster_res0.3') +
    scale_color_manual(values = main_celltype_colors, limits = c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')) +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 12)
mainmarkers <- c('Cd3g', 'Cd4', 'Cd8a', # Conventional T
                 'Trdc','Trdv1','Trgv9', #gdT
                 'Trac', 'Klrb1c', #NKT
                 'Nkg7', 'Ncr1',#Nk
                 'Cd79a', "Ms4a1", #B
                 'Jchain', 'Ighg1', #Plasma
                 "Xcr1", "Cadm1", 'Clec9a', 'Cd24a', #cDC1
                 'Itgax', 'H2-DMb2', #cDC2
                 "Fscn1", "Ccr7", "Cacnb3", #mDC
                 "Siglech", "Ccr9", "Pacsin1", #pDC
                 'Csf1r', #Mono_Mac
                 "Ly6c2", "Ccr2" , "Cd14", "Trem1", #Classical Mono
                 'Cx3cr1', "Itgal", "Ace", "Fcgr3", #NonClassical Mono
                 'Mafb', 'C1qb', #Mac
                 "S100a8", 'S100a9', 'Csf3r', #Neutrophil
                 "Il6", "Fcer1a", "Cpa3", "Ms4a2", "Gata2", #Basophil
                 'Mki67' #Prolif.
                )


DotPlot(SAGA_TIL_MC38_SeuratObj, features = mainmarkers, group.by = 'Maincluster_res0.3', scale = T) + 
    scale_color_gradient(low = '#fde9eb', high = '#db2531') + #FF0000 #db2531
    scale_y_discrete(limits = c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')) +
#    scale_size_area(max_size = 1) +
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 90),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_MarkerGene_DotPlot.pdf'), device = 'pdf', width = 15, height = 6, bg = 'transparent')

In [ ]:
SAGA_TIL_MC38_SeuratObj

In [ ]:
saveRDS(SAGA_TIL_MC38_SeuratObj, paste0(wd, '/objects_v2/SAGA_TIL_MC38_SeuratObj.rds'))

In [ ]:
SAGA_TIL_MC38_SeuratObj <- readRDS(paste0(wd, '/objects_v2/SAGA_TIL_MC38_SeuratObj.rds'))

## Differential abundance analysis

### Direct ratio

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_TIL_B16_SeuratObj$`sgRNA` <-  factor(SAGA_TIL_B16_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_TIL_B16_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Maincluster_res0.3', pt.size = 0.0001) +
    scale_color_manual(values = main_celltype_colors, limits = c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')) +
    ggtitle(label = 'B16F10 Cell Line (Experiment1 + Experiment2)')

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_UMAP.pdf'), device = 'pdf', width = 12, height = 5, bg = 'transparent')
ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_UMAP.png'), device = 'png', width = 12, height = 5, dpi = 300, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_TIL_B16_SeuratObj$`sgRNA` <-  factor(SAGA_TIL_B16_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_TIL_B16_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'ExperimentID', pt.size = 0.0001) +
    ggtitle(label = 'B16F10 Cell Line (Experiment1(10x) + Experiment2(BD))')

In [ ]:
SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3` %>% unique()

In [ ]:
#SAGA_TIL_B16_SeuratObj; Proportion
options(repr.plot.height = 5, repr.plot.width = 15)

cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3` <- factor(SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3`, levels = cell_types)
custom_labeller <- function(variable, value) {
  # 创建一个名为 variable 的列表，用于更改标签显示名称
  labels <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Mono.', 'Macro.','Neutro.', 'Baso.')  # 你可以按需要修改
  names(labels) <- cell_types  # 对应的分面标签

  return(labels[as.character(value)])  # 返回相应的标签
}

FetchData(SAGA_TIL_B16_SeuratObj, vars = c('sgRNA', 'Maincluster_res0.3')) %>%
    group_by(sgRNA, Maincluster_res0.3) %>%
    summarise(n_cell_cluster = n()) %>%
    mutate(n_cell_total = sum(n_cell_cluster), proportion = n_cell_cluster/n_cell_total * 100) %>%
    ggplot(mapping = aes(x = sgRNA, y = proportion, fill = sgRNA)) +
    geom_col(position = 'dodge', alpha = 0.5) +
    facet_wrap(~ Maincluster_res0.3, nrow = 1, scales = 'fixed', strip.position = "top", labeller = custom_labeller) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Proportion Relative to All Immune Cell') +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = c(0.97, 0.5),
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_Relative_Proportion_BarPlot.pdf'), device = 'pdf', width = 15, height = 5, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 12)
SAGA_TIL_MC38_SeuratObj$`sgRNA` <-  factor(SAGA_TIL_MC38_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_TIL_MC38_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Maincluster_res0.3') +
    scale_color_manual(values = main_celltype_colors, limits = c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')) +
    ggtitle(label = 'MC38 Cell Line (Experiment3 + Experiment4)')

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_UMAP.pdf'), device = 'pdf', width = 8, height = 5, bg = 'transparent')
ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_UMAP.png'), device = 'png', width = 8, height = 5, dpi = 300, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 12)
SAGA_TIL_MC38_SeuratObj$`sgRNA` <-  factor(SAGA_TIL_MC38_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_TIL_MC38_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'ExperimentID') +
    ggtitle(label = 'MC38 Cell Line (Experiment3(HXF) + Experiment4(JFF))')

In [ ]:
#SAGA_TIL_MC38_SeuratObj; Proportion
options(repr.plot.height = 7, repr.plot.width = 10)
FetchData(SAGA_TIL_MC38_SeuratObj, vars = c('sgRNA', 'Maincluster_res0.3')) %>%
    group_by(sgRNA, Maincluster_res0.3) %>%
    summarise(n_cell_cluster = n()) %>%
    mutate(n_cell_total = sum(n_cell_cluster), proportion = n_cell_cluster/n_cell_total * 100) %>%
    ggplot(mapping = aes(x = sgRNA, y = proportion, fill = sgRNA)) +
    geom_col(position = 'dodge', alpha = 0.5) +
    facet_wrap(~ Maincluster_res0.3, nrow = 3, scales = 'free', strip.position = "top", labeller = custom_labeller) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Proportion Relative to All Immune Cell') +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = c(0.9, 0.2),
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_Relative_Proportion_BarPlot.pdf'), device = 'pdf', width = 10, height = 7, bg = 'transparent')

## Differential state analysis

### Gene

In [ ]:
Pro_inflammatory_genes <- c('H2-Ab1', 'Ccl5', 'Cxcl9', 'Cxcl10')
Suppressive_genes <- c('Arg1', 'Nos2', 'Cxcl3', 'Cxcl2', 'Csf1r')
T_cell_activation_effector_genes <- c('Ltb', 'Klrk1', 'Cd69', 'Ifng', 'Gzmb')

SAGA_TIL_B16_SeuratObj$`Celltype_sgRNA` <- paste0(SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3`, '_',SAGA_TIL_B16_SeuratObj$`sgRNA`)

cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
sgRNAs <- c("sgNT", "sgC", "sgS")  # 你想要的顺序

desired_order <- unlist(lapply(cell_types, function(ct) paste0(ct, "_", sgRNAs)))

# 转换为因子并设定顺序
SAGA_TIL_B16_SeuratObj$`Celltype_sgRNA` <- factor(
  SAGA_TIL_B16_SeuratObj$`Celltype_sgRNA`,
  levels = desired_order
)

options(repr.plot.height = 15, repr.plot.width = 12)                               
DotPlot(SAGA_TIL_B16_SeuratObj, features = c(Pro_inflammatory_genes, Suppressive_genes, T_cell_activation_effector_genes), group.by = 'Celltype_sgRNA') +
    theme(axis.text = element_text(size = 15, face = "bold"), axis.text.x = element_text(angle = 270))

In [ ]:

SAGA_TIL_MC38_SeuratObj$`Celltype_sgRNA` <- paste0(SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3`, '_',SAGA_TIL_MC38_SeuratObj$`sgRNA`)

cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
sgRNAs <- c("sgNT", "sgC")  # 你想要的顺序

desired_order <- unlist(lapply(cell_types, function(ct) paste0(ct, "_", sgRNAs)))

# 转换为因子并设定顺序
SAGA_TIL_MC38_SeuratObj$`Celltype_sgRNA` <- factor(
  SAGA_TIL_MC38_SeuratObj$`Celltype_sgRNA`,
  levels = desired_order
)

options(repr.plot.height = 15, repr.plot.width = 12)                               
DotPlot(SAGA_TIL_MC38_SeuratObj, features = c(Pro_inflammatory_genes, Suppressive_genes, T_cell_activation_effector_genes), group.by = 'Celltype_sgRNA') +
    theme(axis.text = element_text(size = 15, face = "bold"), axis.text.x = element_text(angle = 270))

### Gene set

#### IFN

##### IFN score

In [ ]:
OrderGene_GSEA <- function(DEGs_wilcox_df) {
    
    DEGs_wilcox_df_list<-DEGs_wilcox_df[['avg_log2FC']] # 提取排序值 only about thousands genes (some genes are filtered due to low expression pct in wilcox test)
    
    if ('gene' %in% colnames(DEGs_wilcox_df)) {
        names(DEGs_wilcox_df_list)<-DEGs_wilcox_df[['gene']] #gene列定义为基因名
    } else{names(DEGs_wilcox_df_list)<-rownames(DEGs_wilcox_df)} #列名定义为基因名)
           
    DEGs_wilcox_df_list <- sort(DEGs_wilcox_df_list, decreasing = T) #按排序值进行排序，制作基因列表

    return(DEGs_wilcox_df_list)
}

In [ ]:
msigdb_Hallmark <- msigdbr(species = 'Mus musculus', category = "H")
msigdb_Hallmark_list <- msigdb_Hallmark %>% split(x = .$gene_symbol, f = .$gs_name)

In [ ]:
Hallmark_GS_selected <- c('HALLMARK_INTERFERON_ALPHA_RESPONSE', 'HALLMARK_INTERFERON_GAMMA_RESPONSE')

In [ ]:
for (GS in Hallmark_GS_selected) {
    SAGA_TIL_B16_SeuratObj <- AddModuleScore(object = SAGA_TIL_B16_SeuratObj, features = list(msigdb_Hallmark_list[[GS]] %>% unique()), name = paste0(GS, '_'), nbin = 50)
    SAGA_TIL_MC38_SeuratObj <- AddModuleScore(object = SAGA_TIL_MC38_SeuratObj, features = list(msigdb_Hallmark_list[[GS]] %>% unique()), name = paste0(GS, '_'), nbin = 50)
}

In [ ]:
#SAGA_TIL_B16_SeuratObj; HALLMARK_INTERFERON_ALPHA_RESPONSE_1
options(repr.plot.width = 20, repr.plot.height = 8)
cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3` <- factor(SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3`, levels = cell_types)
# 创建一个自定义 labeller
custom_labeller <- function(variable, value) {
  # 创建一个名为 variable 的列表，用于更改标签显示名称
  labels <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Mono.', 'Macro.','Neutro.', 'Baso.')  # 你可以按需要修改
  names(labels) <- cell_types  # 对应的分面标签

  return(labels[as.character(value)])  # 返回相应的标签
}


FetchData(SAGA_TIL_B16_SeuratObj, vars = c('Maincluster_res0.3', 'sgRNA', 'HALLMARK_INTERFERON_ALPHA_RESPONSE_1', 'HALLMARK_INTERFERON_GAMMA_RESPONSE_1')) %>%
    ggplot(mapping = aes(x = sgRNA, y = HALLMARK_INTERFERON_ALPHA_RESPONSE_1, fill = sgRNA)) +
    geom_violin(linewidth=0, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=T, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ Maincluster_res0.3, nrow = 1, strip.position = "top", labeller = custom_labeller) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 8, comparisons = list(c("sgS", "sgNT"), c("sgC", "sgNT"))) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Interferon Alpha Response Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_Interferon_Alpha_Response_Score_VlnPlot.pdf'), device = 'pdf', width = 20, height = 9, bg = 'transparent')

In [ ]:
#SAGA_TIL_B16_SeuratObj; HALLMARK_INTERFERON_GAMMA_RESPONSE_1
options(repr.plot.width = 20, repr.plot.height = 8)
FetchData(SAGA_TIL_B16_SeuratObj, vars = c('Maincluster_res0.3', 'sgRNA', 'HALLMARK_INTERFERON_ALPHA_RESPONSE_1', 'HALLMARK_INTERFERON_GAMMA_RESPONSE_1')) %>%
    ggplot(mapping = aes(x = sgRNA, y = HALLMARK_INTERFERON_GAMMA_RESPONSE_1, fill = sgRNA)) +
    geom_violin(linewidth=0, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=T, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ Maincluster_res0.3, nrow = 1, strip.position = "top", labeller = custom_labeller) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 8, comparisons = list(c("sgS", "sgNT"), c("sgC", "sgNT"))) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Interferon Gamma Response Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_Interferon_Gamma_Response_Score_VlnPlot.pdf'), device = 'pdf', width = 20, height = 9, bg = 'transparent')

In [ ]:
#SAGA_TIL_MC38_SeuratObj; HALLMARK_INTERFERON_ALPHA_RESPONSE_1
options(repr.plot.width = 20, repr.plot.height = 8)
cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3` <- factor(SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3`, levels = cell_types)


FetchData(SAGA_TIL_MC38_SeuratObj, vars = c('Maincluster_res0.3', 'sgRNA', 'HALLMARK_INTERFERON_ALPHA_RESPONSE_1', 'HALLMARK_INTERFERON_GAMMA_RESPONSE_1')) %>%
    ggplot(mapping = aes(x = sgRNA, y = HALLMARK_INTERFERON_ALPHA_RESPONSE_1, fill = sgRNA)) +
    geom_violin(linewidth=0, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=F, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ Maincluster_res0.3, nrow = 1, strip.position = "top", labeller = custom_labeller) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 8, comparisons = list(c("sgC", "sgNT"))) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Interferon Alpha Response Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_Interferon_Alpha_Response_Score_VlnPlot.pdf'), device = 'pdf', width = 18, height = 9, bg = 'transparent')

In [ ]:
#SAGA_TIL_MC38_SeuratObj; HALLMARK_INTERFERON_GAMMA_RESPONSE_1
options(repr.plot.width = 20, repr.plot.height = 8)
cell_types <- c('CD8T', 'CD4T', 'gdT', 'NKT', 'NK', 'B_Cell', 'cDC1', 'cDC2', 'mregDC', 'pDC', 'Monocyte', 'Macrophage','Neutrophil', 'Basophil')
SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3` <- factor(SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3`, levels = cell_types)


FetchData(SAGA_TIL_MC38_SeuratObj, vars = c('Maincluster_res0.3', 'sgRNA', 'HALLMARK_INTERFERON_ALPHA_RESPONSE_1', 'HALLMARK_INTERFERON_GAMMA_RESPONSE_1')) %>%
    ggplot(mapping = aes(x = sgRNA, y = HALLMARK_INTERFERON_GAMMA_RESPONSE_1, fill = sgRNA)) +
    geom_violin(linewidth=0, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=F, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ Maincluster_res0.3, nrow = 1, strip.position = "top", labeller = custom_labeller) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 8, comparisons = list(c("sgC", "sgNT"))) +
    scale_x_discrete(labels = NULL) +
    scale_fill_manual(values = sgRNA_color) +
    xlab(label = NULL) +
    ylab(label = 'Interferon Gamma Response Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_Interferon_Gamma_Response_Score_VlnPlot.pdf'), device = 'pdf', width = 18, height = 9, bg = 'transparent')

##### IFN GSEA

In [ ]:
##find DEGs between sgC,sgS and sgNT with wilcox_test; SAGA_TIL_B16_SeuratObj
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$`sgRNA`
DEGs_B16_Imm_sgC_sgNT_wilcox <- FindMarkers(object = SAGA_TIL_B16_SeuratObj, ident.1 = "sgC", ident.2 = 'sgNT', test.use = "wilcox", logfc.threshold = 0, min.pct = 0.1)
DEGs_B16_Imm_sgS_sgNT_wilcox <- FindMarkers(object = SAGA_TIL_B16_SeuratObj, ident.1 = "sgS", ident.2 = 'sgNT', test.use = "wilcox", logfc.threshold = 0, min.pct = 0.1)

In [ ]:
##find DEGs between sgC,sgS and sgNT with wilcox_test; SAGA_TIL_MC38_SeuratObj
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$`sgRNA`
DEGs_MC38_Imm_sgC_sgNT_wilcox <- FindMarkers(object = SAGA_TIL_MC38_SeuratObj, ident.1 = "sgC", ident.2 = 'sgNT', test.use = "wilcox", logfc.threshold = 0, min.pct = 0.1)

In [ ]:
saveRDS(DEGs_B16_Imm_sgC_sgNT_wilcox, paste0(wd, '/objects_v2/DEGs_B16_Imm_sgC_sgNT_wilcox.rds'))
saveRDS(DEGs_B16_Imm_sgS_sgNT_wilcox, paste0(wd, '/objects_v2/DEGs_B16_Imm_sgS_sgNT_wilcox.rds'))
saveRDS(DEGs_MC38_Imm_sgC_sgNT_wilcox, paste0(wd, '/objects_v2/DEGs_MC38_Imm_sgC_sgNT_wilcox.rds'))

In [ ]:
DEGs_B16_Imm_sgC_sgNT_wilcox <- readRDS(paste0(wd, '/objects_v2/DEGs_B16_Imm_sgC_sgNT_wilcox.rds'))
DEGs_B16_Imm_sgS_sgNT_wilcox <- readRDS(paste0(wd, '/objects_v2/DEGs_B16_Imm_sgS_sgNT_wilcox.rds'))
DEGs_MC38_Imm_sgC_sgNT_wilcox <- readRDS(paste0(wd, '/objects_v2/DEGs_MC38_Imm_sgC_sgNT_wilcox.rds'))

In [ ]:
#fgsea
fgsea_B16_Imm_sgC_sgNT_wilcox <- fgsea(pathways = msigdb_Hallmark_list[Hallmark_GS_selected], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgC_sgNT_wilcox), nproc = 1)
fgsea_B16_Imm_sgS_sgNT_wilcox <- fgsea(pathways = msigdb_Hallmark_list[Hallmark_GS_selected], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgS_sgNT_wilcox), nproc = 1)
fgsea_MC38_Imm_sgC_sgNT_wilcox <- fgsea(pathways = msigdb_Hallmark_list[Hallmark_GS_selected], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_MC38_Imm_sgC_sgNT_wilcox), nproc = 1)

In [ ]:
fgsea_B16_Imm_sgC_sgNT_wilcox

In [ ]:
fgsea_B16_Imm_sgS_sgNT_wilcox

In [ ]:
fgsea_MC38_Imm_sgC_sgNT_wilcox

In [ ]:
#INTERFERON_ALPHA
EnrichData_B16_Imm_sgC_sgNT_IFNA <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[1]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgC_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgC', IFN = 'IFNA')

EnrichData_B16_Imm_sgS_sgNT_IFNA <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[1]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgS_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgS', IFN = 'IFNA')

EnrichData_MC38_Imm_sgC_sgNT_IFNA <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[1]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_MC38_Imm_sgC_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgC', IFN = 'IFNA')



#INTERFERON_GAMMA
EnrichData_B16_Imm_sgC_sgNT_IFNG <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[2]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgC_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgC', IFN = 'IFNG')

EnrichData_B16_Imm_sgS_sgNT_IFNG <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[2]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_B16_Imm_sgS_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgS', IFN = 'IFNG')

EnrichData_MC38_Imm_sgC_sgNT_IFNG <- plotEnrichment(pathway = msigdb_Hallmark_list[[Hallmark_GS_selected[2]]], stats = OrderGene_GSEA(DEGs_wilcox_df = DEGs_MC38_Imm_sgC_sgNT_wilcox), ticksSize = 0)$`data` %>%
    mutate(sgRNA = 'sgC', IFN = 'IFNG')

# combine
EnrichData_B16_Imm_sgCS_sgNT <- rbind(EnrichData_B16_Imm_sgC_sgNT_IFNA, EnrichData_B16_Imm_sgS_sgNT_IFNA, EnrichData_B16_Imm_sgC_sgNT_IFNG, EnrichData_B16_Imm_sgS_sgNT_IFNG)
EnrichData_MC38_Imm_sgC_sgNT <- rbind(EnrichData_MC38_Imm_sgC_sgNT_IFNA, EnrichData_MC38_Imm_sgC_sgNT_IFNG)

In [ ]:
# 使用 geom_point 代替 geom_line
options(repr.plot.width = 8, repr.plot.height = 6)
EnrichData_B16_Imm_sgCS_sgNT %>%
    filter(IFN == 'IFNG') %>%
    ggplot(aes(x = x, y = y, fill = sgRNA)) +
    geom_point(size = 3, shape=21, alpha = 0.5, stroke = 0.2) +
    labs(
        title = paste0("Enrichment of Interferon Gamma Response"),
        x = "Rank in Differential Expressed Genes",
        y = "Enrichment Score"
    ) +
    geom_hline(aes(yintercept = 0), linetype = 'dashed') +
    annotate("text", x = Inf, y = Inf, label = "sgC: FDR < 0.012", hjust = 1.287, vjust = 3.5, size = 6) +
    annotate("text", x = Inf, y = Inf, label = "sgS: FDR < 2.369e-8", hjust = 1.1, vjust = 5.5, size = 6) +
    scale_fill_manual(values = sgRNA_color) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_B16_Interferon_Gamma_Response_GSEA_ScatterPlot.pdf'), device = 'pdf', width = 8, height = 6, bg = 'transparent')

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)
ggplot(EnrichData_MC38_Imm_sgC_sgNT, aes(x = x, y = y, fill = sgRNA)) +
    geom_point(size = 3, shape=21, alpha = 0.5, stroke = 0.2) +
    labs(
        title = paste0("Enrichment of Interferon Gamma Response"),
        x = "Rank in Differential Expressed Genes",
        y = "Enrichment Score"
    ) +
    geom_hline(aes(yintercept = 0), linetype = 'dashed') +
    annotate("text", x = Inf, y = Inf, label = "sgC: FDR < 6.260e-14", hjust = 1.1, vjust = 5.5, size = 6) +
    scale_fill_manual(values = sgRNA_color) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'top',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_TIL_MC38_Interferon_Gamma_Response_GSEA_ScatterPlot.pdf'), device = 'pdf', width = 8, height = 6, bg = 'transparent')

In [ ]:
saveRDS(SAGA_TIL_B16_SeuratObj, paste0(wd, '/objects/SAGA_TIL_B16_SeuratObj.rds'))
saveRDS(SAGA_TIL_MC38_SeuratObj, paste0(wd, '/objects/SAGA_TIL_MC38_SeuratObj.rds'))

In [ ]:
SAGA_TIL_B16_SeuratObj <- readRDS(paste0(wd, '/objects/SAGA_TIL_B16_SeuratObj.rds'))
SAGA_TIL_MC38_SeuratObj <- readRDS(paste0(wd, '/objects/SAGA_TIL_MC38_SeuratObj.rds'))

In [ ]:
SAGA_TIL_B16_SeuratObj

In [ ]:
SAGA_TIL_MC38_SeuratObj

## CD8T Cell

### Subset

In [ ]:
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3`
levels(SAGA_TIL_MC38_SeuratObj)

Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3`
levels(SAGA_TIL_B16_SeuratObj)

In [ ]:
# subset CD8T cells for B16
Idents(SAGA_TIL_B16_SeuratObj) <- SAGA_TIL_B16_SeuratObj$`Maincluster_res0.3`
SAGA_CD8T_B16_SeuratObj <- subset(SAGA_TIL_B16_SeuratObj, idents = c('CD8T'))
SAGA_CD8T_B16_SeuratObj

In [ ]:
# subset CD8T cells for MC38
Idents(SAGA_TIL_MC38_SeuratObj) <- SAGA_TIL_MC38_SeuratObj$`Maincluster_res0.3`
SAGA_CD8T_MC38_SeuratObj <- subset(SAGA_TIL_MC38_SeuratObj, idents = c('CD8T'))
SAGA_CD8T_MC38_SeuratObj

### Subclustering and annotation

#### B16

In [ ]:
# UMAP
SAGA_CD8T_B16_SeuratObj <- RunUMAP(SAGA_CD8T_B16_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindNeighbors
SAGA_CD8T_B16_SeuratObj <- FindNeighbors(SAGA_CD8T_B16_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindClusters
options(repr.plot.width = 8, repr.plot.height = 8)
for (i in c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1)) {
  SAGA_CD8T_B16_SeuratObj <- FindClusters(SAGA_CD8T_B16_SeuratObj, resolution = i, verbose = F)
  print(DimPlot(SAGA_CD8T_B16_SeuratObj, reduction = "umap", label = T) + labs(title = paste0("resolution: ", i)))
}

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 8)
FeaturePlot(object = SAGA_CD8T_B16_SeuratObj, features = c("Gzmb"), reduction = 'umap')

In [ ]:
options(repr.plot.height = 28, repr.plot.width = 15)
FeaturePlot(object = SAGA_CD8T_B16_SeuratObj, features = c("Cd4", "Cd8a",'Sell', 'Ccr7', 'Tcf7','Lef1', "Ifng", "Gzmb", "Prf1", "Nkg7", "Tox","Pdcd1", "Ctla4", "Havcr2", "Lag3", "Foxp3", "Pclaf", "Mki67"), reduction = 'umap', ncol = 3)

In [ ]:
# get the multiple_subcluster for c2_res0.1
Idents(SAGA_CD8T_B16_SeuratObj) <- SAGA_CD8T_B16_SeuratObj$`RNA_snn_res.0.1`
SAGA_CD8T_B16_SeuratObj <- FindSubCluster(object = SAGA_CD8T_B16_SeuratObj, cluster = '2', subcluster.name = 'RNA_snn_res.0.1_subclus', graph.name = 'RNA_snn', resolution = 0.1)


In [ ]:
options(repr.plot.height = 8, repr.plot.width = 8)
DimPlot(SAGA_CD8T_B16_SeuratObj, reduction = "umap", label = T, group.by = 'RNA_snn_res.0.1_subclus')

In [ ]:
#find Tmarkers_B16_res0.1_presto with wilcox test in presto
Tmarkers_B16_res0.1_presto <- wilcoxauc(X = SAGA_CD8T_B16_SeuratObj, group_by = 'RNA_snn_res.0.1_subclus')

In [ ]:
Tmarkers_B16_res0.1_presto %>% filter(group=='3', pct_in > 10) %>% arrange(desc(logFC)) %>% slice_max(logFC, n = 50)

In [ ]:
##maincluster RNA_snn_res.0.1
options(repr.plot.height = 10, repr.plot.width = 10)
Idents(SAGA_CD8T_B16_SeuratObj) <- SAGA_CD8T_B16_SeuratObj$`RNA_snn_res.0.1_subclus`

Tcluster_anno <- c('0' = 'CD8T_Ttd', '1' = 'CD8T_Teffector', '2_0' = 'CD8T_Tn', '2_1' = 'CD8T_Tstem', '3' = 'CD8T_Tex')


SAGA_CD8T_B16_SeuratObj$Tcluster_res0.1 <- Tcluster_anno[ as.character(SAGA_CD8T_B16_SeuratObj$`RNA_snn_res.0.1_subclus`) ]


DimPlot(SAGA_CD8T_B16_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Tcluster_res0.1') +
    theme(plot.title = element_text(size = 30),
          legend.text = element_text(size = 20),
          legend.key.size = unit(0.5, "inches")) +
    guides(colour = guide_legend(override.aes = list(size = 5)))

In [ ]:
saveRDS(SAGA_CD8T_B16_SeuratObj, paste0(wd, '/objects_v2/SAGA_CD8T_B16_SeuratObj.rds'))

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 8)
CD8T_markers <- c('Ccr7', 'Sell', 'Tcf7', 'Lef1', 'Slamf6', 'Mki67', 'Ifng', 'Gzma', 'Gzmb', 'Prf1', 'Cd101', 'Cd38', 'Tox', 'Pdcd1', 'Lag3', 'Havcr2')
DotPlot(SAGA_CD8T_B16_SeuratObj, features = CD8T_markers, group.by = 'Tcluster_res0.1', scale = T) + 
    scale_y_discrete(limits = c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd', 'CD8T_Tex'), labels = c('Naive', 'Stem-like', 'Effector-like', 'Terminally \ndifferentiated', 'Exhausted')) +
    scale_color_gradient2(low = '#e0e0e0', mid = '#f7d94c', high = '#db2531') + #FF0000 #db2531 
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 270),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_MarkerGene_DotPlot.pdf'), device = 'pdf', width = 8, height = 4, bg = 'transparent')

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 12)
DimPlot(SAGA_CD8T_B16_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Tcluster_res0.1') +
    scale_color_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd', 'CD8T_Tex'), labels = c('Naive', 'Stem-like', 'Effector-like', 'Terminally differentiated', 'Exhausted')) +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

#### MC38

In [ ]:
# UMAP
SAGA_CD8T_MC38_SeuratObj <- RunUMAP(SAGA_CD8T_MC38_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindNeighbors
SAGA_CD8T_MC38_SeuratObj <- FindNeighbors(SAGA_CD8T_MC38_SeuratObj, dims = 1:50, reduction = "harmony", verbose = F)

In [ ]:
# FindClusters
options(repr.plot.width = 8, repr.plot.height = 8)
for (i in c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1)) {
  SAGA_CD8T_MC38_SeuratObj <- FindClusters(SAGA_CD8T_MC38_SeuratObj, resolution = i, verbose = F)
  print(DimPlot(SAGA_CD8T_MC38_SeuratObj, reduction = "umap", label = T) + labs(title = paste0("resolution: ", i)))
}

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 8)
FeaturePlot(object = SAGA_CD8T_MC38_SeuratObj, features = c("C1qb"), reduction = 'umap')

In [ ]:
options(repr.plot.height = 28, repr.plot.width = 15)
FeaturePlot(object = SAGA_CD8T_MC38_SeuratObj, features = c("Cd4", "Cd8a",'Sell', 'Ccr7', 'Tcf7','Lef1', "Ifng", "Gzmb", "Prf1", "Nkg7", "Tox","Pdcd1", "Ctla4", "Havcr2", "Lag3", "Foxp3", "Pclaf", "Mki67"), reduction = 'umap', ncol = 3)

In [ ]:
options(repr.plot.height = 8, repr.plot.width = 8)
DimPlot(SAGA_CD8T_MC38_SeuratObj, reduction = "umap", label = T, group.by = 'RNA_snn_res.0.5')

In [ ]:
#find Tmarkers_MC38_res0.5_presto with wilcox test in presto
Tmarkers_MC38_res0.5_presto <- wilcoxauc(X = SAGA_CD8T_MC38_SeuratObj, group_by = 'RNA_snn_res.0.5')

In [ ]:
Tmarkers_MC38_res0.5_presto %>% filter(group=='0', pct_in > 10) %>% arrange(desc(logFC)) %>% slice_max(logFC, n = 50)

In [ ]:
##maincluster RNA_snn_res.0.5
options(repr.plot.height = 10, repr.plot.width = 10)
Idents(SAGA_CD8T_MC38_SeuratObj) <- SAGA_CD8T_MC38_SeuratObj$`RNA_snn_res.0.5`

Tcluster_anno <- c('0' = 'Contamination', '1' = 'CD8T_Ttd', '2' = 'CD8T_Ttd','3' = 'CD8T_Tstem', '4' = 'CD8T_Teffector')


SAGA_CD8T_MC38_SeuratObj$Tcluster_res0.5 <- Tcluster_anno[ as.character(SAGA_CD8T_MC38_SeuratObj$`RNA_snn_res.0.5`) ]


DimPlot(SAGA_CD8T_MC38_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Tcluster_res0.5') +
    theme(plot.title = element_text(size = 30),
          legend.text = element_text(size = 20),
          legend.key.size = unit(0.5, "inches")) +
    guides(colour = guide_legend(override.aes = list(size = 5)))

In [ ]:
SAGA_CD8T_MC38_SeuratObj <- subset(SAGA_CD8T_MC38_SeuratObj, subset = CellLine == 'MC38' & Tcluster_res0.5 != 'Contamination')

In [ ]:
DimPlot(SAGA_CD8T_MC38_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Tcluster_res0.5') +
    theme(plot.title = element_text(size = 30),
          legend.text = element_text(size = 20),
          legend.key.size = unit(0.5, "inches")) +
    guides(colour = guide_legend(override.aes = list(size = 5)))

In [ ]:
saveRDS(SAGA_CD8T_MC38_SeuratObj, paste0(wd, '/objects_v2/(SAGA_CD8T_MC38_SeuratObj.rds'))

In [ ]:
options(repr.plot.height = 10, repr.plot.width = 12)
DimPlot(SAGA_CD8T_MC38_SeuratObj, label = T, pt.size = 0.5, label.size = 5, repel = T, reduction = 'umap', group.by = 'Tcluster_res0.5') +
    scale_color_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd'), labels = c('Naive', 'Stem-like', 'Effector-like', 'Terminally differentiated')) +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 8)
CD8T_markers <- c('Ccr7', 'Sell', 'Tcf7', 'Lef1', 'Slamf6', 'Mki67', 'Ifng', 'Gzma', 'Gzmb', 'Prf1', 'Cd101', 'Cd38', 'Tox', 'Pdcd1', 'Lag3', 'Havcr2')
DotPlot(SAGA_CD8T_MC38_SeuratObj, features = CD8T_markers, group.by = 'Tcluster_res0.5', scale = T) + 
    scale_y_discrete(limits = c('CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd'), labels = c('Stem-like', 'Effector-like', 'Terminally \ndifferentiated')) +
    scale_color_gradient2(low = '#e0e0e0', mid = '#f7d94c', high = '#db2531') + #FF0000 #db2531 
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 270),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_MarkerGene_DotPlot.pdf'), device = 'pdf', width = 8, height = 4, bg = 'transparent')

### Differential abundance analysis

#### Direct ratio

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_B16_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_B16_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_CD8T_B16_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Tcluster_res0.1', pt.size = 0.0001) +
    scale_color_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd', 'CD8T_Tex')) +
    ggtitle(label = 'B16F10 Cell Line (Experiment1 + Experiment2)')

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_UMAP.pdf'), device = 'pdf', width = 12, height = 5, bg = 'transparent')
ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_UMAP.png'), device = 'png', width = 12, height = 5, dpi = 300, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_B16_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_B16_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_CD8T_B16_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Tcluster_res0.1') +
    ggtitle(label = 'B16F10 Cell Line CD8+ T Cells (Experiment1 + Experiment2)')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_B16_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_B16_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC', 'sgS'))
DimPlot(object = SAGA_CD8T_B16_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'ExperimentID') +
    ggtitle(label = 'B16F10 Cell Line CD8+ T Cells (Experiment1 + Experiment2)')

In [ ]:
#SAGA_CD8T_B16_SeuratObj; Proportion
options(repr.plot.height = 5, repr.plot.width = 6)
FetchData(SAGA_CD8T_B16_SeuratObj, vars = c('sgRNA', 'Tcluster_res0.1')) %>%
    group_by(sgRNA, Tcluster_res0.1) %>%
    summarise(n_cell_cluster = n()) %>%
    mutate(n_cell_total = sum(n_cell_cluster), proportion = n_cell_cluster/n_cell_total) %>%
    mutate(Cluster = factor(Tcluster_res0.1, levels = rev(c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd', 'CD8T_Tex')))) %>%
    ggplot(mapping = aes(x = sgRNA, y = proportion, fill = Cluster)) +
    geom_col(position = 'stack', alpha = 1) +
    scale_x_discrete(limits = c('sgNT', 'sgC', 'sgS')) +
    scale_fill_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tn', 'CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd', 'CD8T_Tex'), labels = c('Naive', 'Stem-like', 'Effector-like', 'Terminally \ndifferentiated', 'Exhausted')) +
    xlab(label = NULL) +
    ylab(label = 'Fraction') +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_Proportion_BarPlot.pdf'), device = 'pdf', width = 6, height = 5, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_MC38_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_MC38_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC'))
DimPlot(object = SAGA_CD8T_MC38_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Tcluster_res0.5', pt.size = 2) +
    scale_color_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd')) +
    ggtitle(label = 'MC38 Cell Line (Experiment1 + Experiment2)')

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_UMAP.pdf'), device = 'pdf', width = 12, height = 5, bg = 'transparent')
ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_UMAP.png'), device = 'png', width = 12, height = 5, dpi = 300, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_MC38_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_MC38_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC'))
DimPlot(object = SAGA_CD8T_MC38_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'Tcluster_res0.5') +
    ggtitle(label = 'MC38 Cell Line CD8+ T Cells (Experiment3 + Experiment4)')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 16)
SAGA_CD8T_MC38_SeuratObj$`sgRNA` <-  factor(SAGA_CD8T_MC38_SeuratObj$`sgRNA`, levels = c('sgNT', 'sgC'))
DimPlot(object = SAGA_CD8T_MC38_SeuratObj, reduction = 'umap', split.by = 'sgRNA', group.by = 'ExperimentID') +
    ggtitle(label = 'MC38 Cell Line CD8+ T Cells (Experiment3 + Experiment4)')

In [ ]:
#SAGA_CD8T_MC38_SeuratObj; Proportion
options(repr.plot.height = 5, repr.plot.width = 6)
FetchData(SAGA_CD8T_MC38_SeuratObj, vars = c('sgRNA', 'Tcluster_res0.5')) %>%
    group_by(sgRNA, Tcluster_res0.5) %>%
    summarise(n_cell_cluster = n()) %>%
    mutate(n_cell_total = sum(n_cell_cluster), proportion = n_cell_cluster/n_cell_total) %>%
    mutate(Cluster = factor(Tcluster_res0.5, levels = rev(c('CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd')))) %>%
    ggplot(mapping = aes(x = sgRNA, y = proportion, fill = Cluster)) +
    geom_col(position = 'stack', alpha = 1) +
    scale_x_discrete(limits = c('sgNT', 'sgC')) +
    scale_fill_manual(values = imm_subcelltype_colors, limits = c('CD8T_Tstem', 'CD8T_Teffector', 'CD8T_Ttd'), labels = c('Stem-like', 'Effector-like', 'Terminally \ndifferentiated')) +
    xlab(label = NULL) +
    ylab(label = 'Fraction') +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_Proportion_BarPlot.pdf'), device = 'pdf', width = 6, height = 5, bg = 'transparent')

### Differential state analysis

#### Gene

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 7)

DotPlot(SAGA_CD8T_B16_SeuratObj, features = c('Ccl5', 'Ifng', 'Prf1', 'Gzmb'), group.by = 'sgRNA', scale = F) + 
    scale_y_discrete(limits = c('sgNT', 'sgC', 'sgS')) +
    scale_color_gradient(low = '#fde9eb', high = '#db2531') + #FF0000 #db2531
#    scale_size_area(max_size = 1) +
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_GeneExpression_DotPlot.pdf'), device = 'pdf', width = 7, height = 4, bg = 'transparent')

In [ ]:
options(repr.plot.height = 4, repr.plot.width = 7)

DotPlot(SAGA_CD8T_MC38_SeuratObj, features = c('Ccl5', 'Ifng', 'Prf1', 'Gzmb'), group.by = 'sgRNA', scale = F) + 
    scale_y_discrete(limits = c('sgNT', 'sgC')) +
    scale_color_gradient(low = '#fde9eb', high = '#db2531') + #FF0000 #db2531
#    scale_size_area(max_size = 1) +
    xlab(label = NULL) +
    ylab(label = NULL) +
    theme_classic(base_size = 20, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'right',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_GeneExpression_DotPlot.pdf'), device = 'pdf', width = 7, height = 4, bg = 'transparent')

#### Gene set

##### CD8_Related

In [ ]:
Effector_cytokine_genes <- c(
  "Ctla4", "Xcl1", "Gadd45b", "Ifng", "Lif", "Ccl4", "Il3", "Ccl3", 
  "Irf4", "Serpinb6b", "Serpinb9", "Gzmc", "Gzmb", "Nr4a1", "Cd200", 
  "Il2ra", "Il2", "Il21", "Nr4a3", "Tnfrsf9", "Mfsd2", "Cxcl10", 
  "Hk2", "Utf1", "Irf8", "Mela", "Sema7a", "Serpine2"
)

Early_activation_genes_human <- c(
    "CXCL13", "HAVCR2", "CTLA4", "TOX", "TNFRSF9", "DTHD1", "NAB1",
    "TTN", "PDCD1", "IPCEF1", "ITGAE", "FKBP5", "SIRPG", "FCRL3", "CD84",
    "PAM", "CXCR6", "ETS1", "ATHL1", "CD27", "PYHIN1", "IL6ST", "SMAP2",
    "LYST", "ERAP2", "KLRC4", "LRMP", "CCL4L2", "PRDM1", "AHI1", "PLCB2",
    "EVL", "LPIN1", "PDE4D", "RBPJ", "TCIRG1", "OXNAD1", "CBLB", "HNRNPLL",
    "DUSP4", "NAP1L4", "TXNIP", "RGS1", "CCL4L1", "RAP1GDS1", "TIGIT",
    "PARK7", "CCL4", "IKZF3", "SH2D1A", "ATXN1", "ITK")

Early_activation_genes_mouse <- c(
    "Cxcl13", "Havcr2", "Ctla4", "Tox", "Tnfrsf9", "Dthd1", "Nab1",
    "Ttn", "Pdcd1", "Ipcef1", "Itgae", "Fkbp5", "Sirpg", "Fcrl3", "Cd84",
    "Pam", "Cxcr6", "Ets1", "Cd27", "Il6st", "Smap2", "Lyst", "Erap2",
    "Klrc4", "Lrmp", "Ccl4", "Prdm1", "Ahi1", "Plcb2", "Evl", "Lpin1",
    "Pde4d", "Rbpj", "Tcirg1", "Oxnad1", "Cblb", "Hnrnpll", "Dusp4",
    "Nap1l4", "Txnip", "Rgs1", "Rap1gds1", "Tigit", "Park7", "Ikzf3",
    "Sh2d1a", "Atxn1", "Itk"
)

In [ ]:
Curated_GS <- c('Effector_or_cytokine_production', 'Early_activation', 'Program_A', 'Program_B', 'Program_C', 'Program_D')
Curated_GS_list <- list('Effector_or_cytokine_production' = Effector_cytokine_genes, 'Early_activation' = Early_activation_genes_mouse)

In [ ]:
for (GS in Curated_GS) {
    SAGA_CD8T_B16_SeuratObj <- AddModuleScore(object = SAGA_CD8T_B16_SeuratObj, features = list(Curated_GS_list[[GS]] %>% unique()), name = paste0(GS, '_'))
    SAGA_CD8T_MC38_SeuratObj <- AddModuleScore(object = SAGA_CD8T_MC38_SeuratObj, features = list(Curated_GS_list[[GS]] %>% unique()), name = paste0(GS, '_'))
}

In [ ]:
SAGA_CD8T_B16_SeuratObj[[]] %>% colnames()

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 8)

FetchData(SAGA_CD8T_B16_SeuratObj, vars = c('sgRNA', 'Early_activation_1', 'Effector_or_cytokine_production_1')) %>%
    pivot_longer(cols = c('Early_activation_1', 'Effector_or_cytokine_production_1'), names_to = 'geneset', values_to = 'score') %>%
    ggplot(mapping = aes(x = sgRNA, y = score, fill = sgRNA)) +
    geom_violin(linewidth=0.2, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=T, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ geneset, nrow = 1, strip.position = 'top', scales = 'free', labeller = as_labeller(c(Early_activation_1 = 'Early activation', Effector_or_cytokine_production_1 = 'Effector/cytokine \nproduction'))) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 6, comparisons = list(c("sgC", "sgNT"), c("sgS", "sgNT"))) +
    scale_x_discrete(limits = c('sgNT', 'sgC', 'sgS')) +
    scale_fill_manual(values = sgRNA_color, limits = c('sgNT', 'sgC', 'sgS')) +
    xlab(label = NULL) +
    ylab(label = 'Activity Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'none',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_B16_Activity_Score_VlnPlot.pdf'), device = 'pdf', width = 8, height = 6, bg = 'transparent')

In [ ]:
options(repr.plot.height = 6, repr.plot.width = 8)

FetchData(SAGA_CD8T_MC38_SeuratObj, vars = c('sgRNA', 'Early_activation_1', 'Effector_or_cytokine_production_1')) %>%
    pivot_longer(cols = c('Early_activation_1', 'Effector_or_cytokine_production_1'), names_to = 'geneset', values_to = 'score') %>%
    ggplot(mapping = aes(x = sgRNA, y = score, fill = sgRNA)) +
    geom_violin(linewidth=0.2, alpha = 0.5, position = position_dodge(0.85)) +
    geom_boxplot(notch=T, width=0.2, linewidth=0.2, alpha = 0.5, position = position_dodge(0.85), outlier.size = 0) +
    facet_wrap(~ geneset, nrow = 1, strip.position = 'top', scales = 'free', labeller = as_labeller(c(Early_activation_1 = 'Early activation', Effector_or_cytokine_production_1 = 'Effector/cytokine \nproduction'))) +
    stat_compare_means(mapping = aes(group = sgRNA), label = 'p.signif', method = 'wilcox.test', size = 6, comparisons = list(c("sgC", "sgNT"))) +
    scale_x_discrete(limits = c('sgNT', 'sgC')) +
    scale_fill_manual(values = sgRNA_color, limits = c('sgNT', 'sgC')) +
    xlab(label = NULL) +
    ylab(label = 'Activity Score') +
    theme_classic(base_size = 25, base_family = 'Helvetica', base_line_size = 0.5, base_rect_size = 0) +
    transparent_bg +
    theme(legend.position = 'none',
          axis.text = element_text(colour = 'black'),
          axis.text.x = element_text(angle = 0),
          axis.title.x = element_text(margin = margin(15,0,0,0)),
          axis.title.y = element_text(margin = margin(0,15,0,0)))

ggsave(filename = paste0(wd, '/figures_v2', '/', 'SAGA_CD8T_MC38_Activity_Score_VlnPlot.pdf'), device = 'pdf', width = 8, height = 6, bg = 'transparent')

In [ ]:
saveRDS(SAGA_CD8T_B16_SeuratObj, paste0(wd, '/objects/SAGA_CD8T_B16_SeuratObj.rds'))
saveRDS(SAGA_CD8T_MC38_SeuratObj, paste0(wd, '/objects/SAGA_CD8T_MC38_SeuratObj.rds'))

In [ ]:
SAGA_CD8T_B16_SeuratObj <- readRDS(paste0(wd, '/objects/SAGA_CD8T_B16_SeuratObj.rds'))
SAGA_CD8T_MC38_SeuratObj <- readRDS(paste0(wd, '/objects/SAGA_CD8T_MC38_SeuratObj.rds'))